In [2]:
print("Hello")

Hello


# install libraries

In [11]:
# pip install apify-client
# !pip install textblob
# pip install Praw
# pip install TikTokApi 

# Extract data from reddddit

In [48]:
import praw
import pandas as pd
reddit = praw.Reddit(
    client_id="h7SCVUWMMkrKxa6dW7K6aA",
    client_secret="ndIj2lKITQBJ6NzHAD10FkyuPZer2A",
    user_agent="my_app"
)

# Example: fetch top posts from r/electronic_cigarette
subreddit = reddit.subreddit("VapeWild")

rows = []
for post in subreddit.top(limit=50):
    row = {
        "datetime": post.created_utc,
        "niche": "vape",
        "product": None,
        "hashtags": None,
        "description": post.title,
        "country": None,
        "area": None,
        "store/channel_name": post.author.name if post.author else None,
        "social_platform": "reddit"
    }
    rows.append(row)

--- make DataFrame ---
df = pd.DataFrame(rows)

# --- convert datetime to readable format ---
df["datetime"] = pd.to_datetime(df["datetime"], unit="s")

# --- show the DataFrame ---
df.head(10)  # print first 10 rows
    


,datetime,niche,product,hashtags,description,country,area,store/channel_name,social_platform
0,2018-11-11 18:04:40,vape,None,None,Vapewild i love you but we need to talk....,None,None,the-jedi,reddit
1,2020-09-10 14:50:13,vape,None,None,So Long and Thanks for All the Fish,None,None,VapeWild,reddit
2,2019-03-23 23:12:28,vape,None,None,7 bottles for $1.10 bless vape wild,None,None,thecrocobear,reddit
3,2018-12-06 20:32:56,vape,None,None,Thanks vapewild!,None,None,Jugglo55,reddit
4,2018-11-20 20:04:48,vape,None,None,Awesome customer service !,None,None,Twrecks_,reddit
5,2018-08-04 13:11:51,vape,None,None,"I love Vape Wild, but there needs to be a way ...",None,None,RandomHero27,reddit
6,2019-09-11 20:24:08,vape,None,None,"In light of recent news, here's a thead for yo...",None,None,VapeWild,reddit
7,2019-01-01 05:31:38,vape,None,None,Just another amazing customer service post.,None,None,captured3,reddit
8,2018-11-14 20:12:14,vape,None,None,"Ordered 60mL, Received 140mL",None,None,Khryzis,reddit
9,2018-10-01 14:36:06,vape,None,None,"Oh boy, we got a lot of stuff today to look at...",None,None,VapeWild,reddit


# Extract relevant information only 

In [8]:
# ----- PRAW: targeted vape data fetch (last 7 days) -----
import re
from datetime import datetime, timedelta, timezone
from zoneinfo import ZoneInfo

import praw

# --- 1) Configure your Reddit credentials ---
# reddit = praw.Reddit(
#     client_id="YOUR_CLIENT_ID",
#     client_secret="YOUR_CLIENT_SECRET",
#     user_agent="my_vape_app/1.0"
# )
reddit = praw.Reddit(
    client_id="h7SCVUWMMkrKxa6dW7K6aA",
    client_secret="ndIj2lKITQBJ6NzHAD10FkyuPZer2A",
    user_agent="my_app"
)

# --- 2) Target subreddits that are actually about vaping ---
VAPE_SUBS = [
         "VapeWild","Juul" # remove subs you don't want
]

# --- 3) Vape keyword filter (title/selftext) ---
KEYWORD_RE = re.compile(
    r"\b(vape|vaping|e-?cig|e\s*cigarette|e-?liquid|ejuice|"
    r"nic(?:otine)?\s*salt|pod(?:\s*system)?|disposable|mod|coil|atomizer|"
    r"lush\s*ice|strawberry\s*kiwi|blue\s*razz|mango\s*ice|elf\s*bar|juul|smok|vaporesso|relx|lost\s*mary)\b",
    flags=re.IGNORECASE
)

# --- 4) Simple flavor/product extraction (extend as you wish) ---
FLAVORS = [
    "Lush Ice", "Strawberry Kiwi", "Blue Razz", "Mango Ice", "Watermelon Ice",
    "Peach Ice", "Grape Ice", "Cool Mint", "Triple Mango", "Pink Lemonade", "Cola Ice"
]
BRANDS = ["Elf Bar", "JUUL", "SMOK", "Vaporesso", "RELX", "HQD", "Lost Mary", "Geek Bar", "OXBAR"]

def extract_product(text):
    hit = None
    for f in FLAVORS:
        if f.lower() in text.lower():
            hit = f
            break
    if not hit:
        for b in BRANDS:
            if b.lower() in text.lower():
                hit = b
                break
    return hit

def extract_hashtags(text):
    # Reddit rarely uses hashtags, but capture them if present
    return re.findall(r"#\w+", text)

def to_pk_time(utc_ts):
    dt = datetime.fromtimestamp(utc_ts, tz=timezone.utc).astimezone(ZoneInfo("Asia/Karachi"))
    return dt.strftime("%d/%m/%Y %H:%M:%S")  # matches your 23/08/2025 style

def fetch_reddit_vape_last_week(limit_per_sub=200):
    now_utc = datetime.now(timezone.utc)
    since_utc = now_utc - timedelta(days=7)
    since_ts = since_utc.timestamp()

    all_rows = []
    seen_ids = set()

    # Lucene query focuses search; we still post-filter with regex
    lucene_query = (
        '(title:(vape OR vaping OR "e cig" OR "e-cig" OR "e liquid" OR ejuice OR disposable OR "nic salt" OR pod)'
        ' OR selftext:(vape OR vaping OR "e cig" OR "e-cig" OR "e liquid" OR ejuice OR disposable OR "nic salt" OR pod))'
    )

    for sub in VAPE_SUBS:
        sr = reddit.subreddit(sub)
        # Search recent; we’ll still enforce the exact time range ourselves:
        for post in sr.search(lucene_query, syntax="lucene", sort="new", limit=limit_per_sub):
            if post.id in seen_ids:
                continue
            seen_ids.add(post.id)

            # Time window: last 7 days
            if not getattr(post, "created_utc", None):
                continue
            if post.created_utc < since_ts:
                continue

            # Text to filter against
            title = post.title or ""
            selftext = getattr(post, "selftext", "") or ""
            combined = f"{title}\n{selftext}"

            # Vape keyword filter
            if not KEYWORD_RE.search(combined):
                continue

            # Build row in your schema
            row = {
                "datetime": to_pk_time(post.created_utc),
                "niche": "vape",
                "product": extract_product(combined),
                "hashtags": extract_hashtags(combined),
                "description": title if len(title) > 5 else (selftext[:180] + "…"),
                "country": None,   # Reddit doesn't provide reliably
                "area": None,
                "store/channel_name": post.author.name if post.author else None,
                "social_platform": "reddit",
            }
            all_rows.append(row)

    return all_rows

if __name__ == "__main__":
    rows = fetch_reddit_vape_last_week()
    for r in rows[:10]:
        print(r)
    print(f"Total rows: {len(rows)}")


Total rows: 0


# Extract data from TikTok

In [24]:
from TikTokApi import TikTokApi

In [27]:
pip install tiktokapipy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
from TikTokApi import TikTokApi